In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../../") 

In [ ]:
import pandas as pd
import numpy as np
import numpyro
numpyro.set_host_device_count(1)

from epimodel import preprocess_data, run_model, candidate_model, EpidemiologicalParameters

In [ ]:
ep = EpidemiologicalParameters()
data = preprocess_data('../../data/all_merged_data.csv')


# I made up some NPI thresholds because I'm lazy. 
# these NPIs are on if there exists a limit, and it is less than 6 people allowed in the respective category
for i in range(4):
    data.active_cms[:, i, :] = np.logical_and(data.active_cms[:, i, :] > 0, data.active_cms[:, i, :] <7) 
    
# these NPIs (the household NPIs) are on if there is any limit on household gatherings at all
for i in range(4):
    # i.e., if there is any limit, since its 0 otherwise
    data.active_cms[:, i+4, :] = data.active_cms[:, i+4, :] > 0
    
data.active_cms[:, 8, :] = data.active_cms[:, 8, :] > 2

In [ ]:
samples, info = run_model(candidate_model, data, ep, num_chains=2)

In [ ]:
from epimodel.plotting import *

In [ ]:
plot_intervention_effectiveness(samples, data.CMs, xlim=[-50, 50])